In [1]:
import numpy as np
import sympy as sym
import math as math
import matplotlib.pyplot as pl

In [34]:
x = sym.Symbol('x',real=True)

def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn
    

def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots
def GetHermiteRecursive(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = (2*x*GetHermiteRecursive(n-1,x))-(2*(n-1)*GetHermiteRecursive(n-2,x))
   
    return sym.expand(poly,x)

def GetDHermite(n,x):
    Poly=GetHermiteRecursive(n,x)
    return sym.diff(Poly,x,1)


def GetAllRootsGHermite(n):
    N= (4*n+1)**(1/2)

    xn = np.linspace(-N,N,300)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermiteRecursive(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

def GetWeightsGHer(n):

    Roots = GetAllRootsGHermite(n)
    

    PolyHermite= sym.lambdify([x],GetHermiteRecursive(n-1,x),'numpy')



    Weights = ((2**(n-1))*math.factorial(n)*((np.pi)**(1/2)))/((n**2)*(PolyHermite(Roots))**2)
    
    return Weights


In [38]:
raices=GetAllRootsGHermite(20)

In [39]:
pesos=GetWeightsGHer(20)

In [42]:
np.polynomial.hermite.hermgauss(20)

(array([-5.38748089, -4.60368245, -3.94476404, -3.34785457, -2.78880606,
        -2.254974  , -1.73853771, -1.23407622, -0.73747373, -0.24534071,
         0.24534071,  0.73747373,  1.23407622,  1.73853771,  2.254974  ,
         2.78880606,  3.34785457,  3.94476404,  4.60368245,  5.38748089]),
 array([2.22939365e-13, 4.39934099e-10, 1.08606937e-07, 7.80255648e-06,
        2.28338636e-04, 3.24377334e-03, 2.48105209e-02, 1.09017206e-01,
        2.86675505e-01, 4.62243670e-01, 4.62243670e-01, 2.86675505e-01,
        1.09017206e-01, 2.48105209e-02, 3.24377334e-03, 2.28338636e-04,
        7.80255648e-06, 1.08606937e-07, 4.39934099e-10, 2.22939365e-13]))

In [48]:
I=0
f= lambda x: 2*np.sqrt(1/(np.pi))*x**4
for i in range(len(pesos)):
    I+=pesos[i]*f(raices[i])
I

1.5000000000066762